### Import Statements
The libraries used are `pandas`, `pathlib`, `imblearn`, and `sklearn`. 

In [358]:
import pandas as pd
from pathlib import Path
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, chi2, SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from skfeature.function.similarity_based import fisher_score


### Import the data and create a dataframe
1. Use `pandas` and `pathlib` to read the `BankChurnersPrimary.csv` CSV from the `Resources` folder.
2. Sample the dataset to review the data.

In [359]:
starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_model/Resources/BankChurnersPrimary.csv"))
starter_df.sample(5)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
9215,789594108,Existing Customer,47,M,5,College,Single,$80K - $120K,Blue,41,...,3429.0,1779,1650.0,0.938,14107,99,0.650,0.519,0.000093,0.99991
4645,711606558,Existing Customer,49,F,1,High School,Married,Less than $40K,Blue,39,...,1909.0,866,1043.0,0.880,4697,83,0.844,0.454,0.000104,0.99990
7383,713509908,Existing Customer,48,F,2,Graduate,Married,Unknown,Blue,35,...,1755.0,1058,697.0,0.697,4719,90,0.800,0.603,0.000033,0.99997
5639,789990333,Existing Customer,43,F,4,Graduate,Single,$40K - $60K,Blue,35,...,2303.0,722,1581.0,0.807,4469,80,0.778,0.314,0.000119,0.99988
31,712991808,Existing Customer,53,M,2,Uneducated,Married,$60K - $80K,Blue,48,...,2451.0,1690,761.0,1.323,1596,26,1.600,0.690,0.000125,0.99988


### Remove any columns that won't be used.
Naive Bayes columns can't be used (it's calculated from features and target).
Education_Level, Marital_Status, and Income_Category have many Nan/Unknown values. Drop to preserve sample size.
CLIENTNUM is irrelevant.

In [360]:
starter_df.drop(columns=[
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'CLIENTNUM',
'Education_Level', 
'Marital_Status', 
'Income_Category'
], 
inplace= True)

### Evaluate the Card Type column for use as a feature
The Card Type data is processed with OneHotEncoder.

In [361]:

enc = OneHotEncoder(sparse=False)
categorical_variables = ['Card_Category']
encoded_data = enc.fit_transform(starter_df[categorical_variables])
encoded_dataframe = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables))
starter_df.drop(columns=['Card_Category'], inplace=True)
starter_df = pd.concat([starter_df, encoded_dataframe.set_axis(starter_df.index)], axis=1)
starter_df.sample(10)


c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Evaluate the Gender column for use as a feature
Gender is a binary classification, so we don't need to OneHotEncode.
The values will be converted to numericals and specified as datatype `int`.
1. Male = 0
2. Female = 1 


In [363]:
starter_df = starter_df.replace({'Gender' : {
'M': 0, 
'F': 1
}})
starter_df.Gender = starter_df.Gender.astype(int)

### Evaluate the Attrition_Flag column for use as the target
Attrition_Flag is a binary classification, so we don't need to OneHotEncode.
The values will be converted to numericals and specified as datatype `int`.
1. Attrited Customer = 0
2. Existing Customer = 1 

In [364]:
starter_df = starter_df.replace({'Attrition_Flag' : {
'Attrited Customer': 0, 
'Existing Customer': 1
}})
starter_df.Attrition_Flag = starter_df.Attrition_Flag.astype(int)

### Define our features(X) and our target(y)

In [366]:
y = starter_df['Attrition_Flag']
X = starter_df.drop(columns=['Attrition_Flag'])

### Split features and target in to training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### StandardScaler will be used to scale the data 
StandardScaler will only be fit to X_train.

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Use SMOTE to add synthetic data and balance our target/feature value count

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

### Create an instance of the GradientBoostingClassifier model

In [ ]:
gbc_model = GradientBoostingClassifier(
n_estimators=500,
random_state= 2
)

### Fit the model on the training data

In [ ]:
gbc_model.fit(X_train, y_train)

### Make predictions on the test data

In [ ]:
gbc_test_predictions = gbc_model.predict(X_test)

### Evaluate the model's performance

In [ ]:
accuracy = accuracy_score(y_test, gbc_test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

### Generate a confusion matrix

In [ ]:
gbc_test_matrix = confusion_matrix(y_test, gbc_test_predictions)
print(gbc_test_matrix)

### Generate a classification report

In [ ]:
gbc_testing_report = classification_report(y_test, gbc_test_predictions)
print(gbc_testing_report)

### Generate an imbalanced classification report

In [ ]:
imb__gbc_testing_report = classification_report_imbalanced(y_test, gbc_test_predictions)
print(imb__gbc_testing_report)